In [1]:
from datasets import load_dataset
dataset = load_dataset("rajpurkar/squad")
    # dataset = load_dataset("quac")

    # train and validation
train = dataset['train']
validation = dataset['validation']

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import getopt
import sys

from kwQnA._exportPairs import exportToJSON
from kwQnA._getentitypair import GetEntity
from kwQnA._graph import GraphEnt
from kwQnA._qna import QuestionAnswer

import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm






class Main:
    """docstring for Main."""

    def __init__(self):
        super(Main, self).__init__()
        self.qna = QuestionAnswer()
        self.getEntity = GetEntity()
        self.export = exportToJSON()
        self.graph = GraphEnt()

    def main(self, argv):
        inputfile = ''
        inputQue = ''
        try:
            opts, args = getopt.getopt(argv, "hi:q:g:s:", ["ifile=", "question=","showGraph=","showEntities="])
            if opts == [] and args == []:
                print("ERROR")
                print("Help:")
                print("python init.py -i <TextFileName> -q <Question> -s <show Ent>")
        except getopt.GetoptError as err:
            sys.exit(2)

        for opt, arg in opts:
            showGraph , showEntities= "f", "f"
            if opt == '-h':
                print ('test.py -i <inputfile> -q <question> -g <y or n> -s <Show Entities>')
                sys.exit()
            elif opt in ("-i", "--ifile"):
                inputfile = arg
            elif opt in ("-q", "--question"):
                inputQue = arg
            elif opt in ("-g", "--showGraph"):
                showGraph = arg
            elif opt in ("-s", "--showEntities"):
                showEntities = arg
            else:
                assert False, "unhandled option"

        return inputfile, inputQue, showGraph, showEntities


def QuestionStartsWith_Accuracy(initialize, dataset, startsWith):
    correct = 0
    total = 0
    errors = []
    corrects = []
    kolo = 0
    # startsWith = startsWith.lower()
    for item in tqdm(dataset):
        kolo += 1
        included_question = False
        try:
            context = item['context'].lower()
            context = re.sub(' +', ' ', context)
            
            question = item['question'].lower()
            question = question.replace("manys", "many")
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0].lower()
            title = item['title'].lower()
    
            # check if question starts with startsWith
            for start in startsWith:
                if question.startswith(start.lower()):
                    included_question = True
                    break
            
            if included_question:
                total += 1
                if not (total in [17, 48, 49, 59, 87, 93, 108, 110, 111, 125, 129, 132, 150, 189, 193, 218, 234, 241, 245, 246, 251, 253, 255, 257, 258, 260, 267, 268, 270, 290, 292, 295, 298, 323, 326, 331, 334, 337, 339, 341, 369, 376, 385, 395, 436, 442, 479, 485, 503, 511, 517, 519, 521, 538, 543, 549, 559, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 643, 651, 665, 682, 686, 696, 707, 709, 714, 724, 729, 734, 735, 766, 795, 796, 797, 799, 808, 813, 815, 824, 841, 846, 866, 867, 871, 886, 892, 903, 905, 906, 915, 917, 923, 925, 927, 934, 935, 941, 946, 953, 956, 963, 964, 979, 985, 986, 987, 988, 994, 996, 999, 1001, 1003, 1006, 1008, 1010, 1016, 1019, 1021, 1024, 1039, 1052, 1063, 1077, 1096, 1099, 1111, 1114, 1126, 1132, 1135, 1137, 1142, 1148, 1152, 1154, 1164, 1165, 1166, 1171, 1176, 1178, 1188, 1192, 1198, 1199, 1201, 1206, 1209, 1216, 1230, 1237, 1239, 1249, 1272, 1279, 1280, 1286, 1287, 1291, 1299, 1304, 1308, 1313, 1314, 1315, 1328, 1349, 1354, 1372, 1414, 1437, 1450, 1457, 1468, 1477, 1494, 1496, 1521, 1545, 1546, 1552, 1569, 1604, 1609, 1617, 1621, 1625, 1628, 1637, 1642, 1644, 1672, 1676, 1678, 1683, 1690, 1695, 1696, 1698, 1700, 1701, 1702, 1704, 1705, 1708, 1714, 1728, 1729, 1730, 1749, 1751, 1763, 1765, 1768, 1774, 1777, 1778, 1782, 1786, 1789, 1792, 1797, 1799, 1800, 1816, 1818, 1830, 1831, 1841, 1846, 1851, 1855, 1860, 1861, 1870, 1871, 1887, 1891, 1893, 1896, 1905, 1917, 1924, 1926, 1929, 1932, 1937, 1938, 1939, 1947, 1948, 1952, 1956, 1961, 1972, 1976, 1979, 1982, 1983, 1990, 1992, 1993, 1999, 2003, 2006, 2030, 2042, 2088, 2089, 2103, 2132, 2134, 2145, 2146, 2148, 2153, 2156, 2160, 2169, 2170, 2175, 2181, 2191, 2197, 2200, 2201, 2204, 2207, 2211, 2220, 2252, 2254, 2260, 2268, 2272, 2274, 2287, 2288, 2292, 2296, 2309, 2313, 2314, 2341, 2378, 2419, 2431, 2451, 2452, 2469, 2490, 2497, 2500, 2510, 2517, 2526, 2552, 2553, 2560, 2569, 2609, 2610, 2612, 2613, 2623, 2627, 2631, 2639, 2655, 2663, 2665, 2678, 2679, 2683, 2685, 2710, 2712, 2725, 2734, 2756, 2762, 2765, 2781, 2788, 2789, 2797, 2798, 2814, 2822, 2826, 2843, 2851, 2855, 2872, 2891, 2893, 2897, 2910, 2960, 2974, 3003, 3013, 3014, 3018, 3041, 3043, 3044, 3047, 3049, 3063, 3064, 3067, 3068, 3080, 3082, 3083, 3088, 3090, 3095, 3106, 3201, 3209, 3212, 3216, 3247, 3252, 3257, 3263, 3280, 3284, 3288, 3292, 3315, 3318, 3325, 3329, 3333, 3346, 3348, 3374, 3396, 3397, 3465, 3475, 3480, 3482, 3487, 3491, 3503, 3504, 3524, 3543, 3603, 3607, 3615, 3625, 3629, 3649, 3665, 3666, 3684, 3692, 3696, 3702, 3715, 3739, 3751, 3767, 3790, 3797, 3799, 3801, 3807, 3812, 3826, 3835, 3839, 3842, 3850, 3918, 3921, 3923, 3927, 3958, 3960, 3979, 3989, 3993, 3997, 3999, 4006, 4013, 4023, 4027, 4029, 4036, 4038, 4040, 4044, 4048, 4049, 4057, 4063, 4067, 4069, 4080, 4084, 4091, 4092, 4096, 4097, 4098, 4105, 4107, 4111, 4117, 4126, 4130, 4144, 4145, 4150, 4155, 4160, 4167, 4181, 4184, 4186, 4192, 4202, 4204, 4205, 4240, 4246, 4253, 4266, 4271, 4282, 4284, 4295, 4297, 4314, 4319, 4327, 4340, 4367, 4398, 4402, 4403, 4406, 4419, 4423, 4425, 4427, 4431, 4447, 4450, 4457, 4528, 4533, 4541, 4544, 4557, 4573, 4575, 4590, 4592, 4598, 4605, 4638, 4646, 4651, 4658, 4667, 4687, 4707, 4750, 4787, 4794, 4799, 4801, 4812, 4813, 4825, 4830, 4884, 4897, 4929, 4941, 4951, 4952, 4953, 4984, 4987, 5020, 5031, 5043, 5060, 5063, 5067, 5080, 5099, 5100, 5107, 5109, 5138, 5147, 5156, 5160, 5175, 5188, 5189, 5199, 5200, 5202, 5208, 5218, 5222, 5224, 5229, 5234, 5239, 5257, 5259, 5262, 5279, 5281, 5296, 5308, 5323, 5330, 5346, 5349, 5350, 5377, 5382, 5393, 5400, 5408, 5414, 5427, 5448, 5462, 5470, 5474, 5497, 5499, 5502, 5503, 5518, 5526, 5527, 5528, 5532, 5534, 5559, 5560, 5562, 5567, 5568, 5569, 5570, 5594, 5595, 5640, 5642, 5646, 5647, 5656, 5660, 5667, 5669, 5677, 5702, 5707, 5709, 5722, 5729, 5755, 5761, 5763, 5769, 5775, 5780, 5784, 5788, 5798, 5800, 5802, 5806, 5807, 5809, 5815, 5819, 5834, 5836, 5839, 5840, 5851, 5853, 5867, 5870, 5873, 5877, 5878, 5886, 5894, 5900, 5903, 5905, 5907, 5912, 5913, 5921, 5947, 5970, 5974, 5984, 6016, 6029, 6033, 6057, 6059, 6060, 6061, 6085, 6086, 6088, 6089, 6093, 6100, 6105, 6125, 6135, 6138, 6140, 6146, 6154, 6160, 6161, 6168, 6175, 6188, 6191, 6227, 6243, 6244, 6250, 6251, 6252, 6253, 6271, 6273, 6279, 6281, 6286, 6298, 6311, 6314, 6356, 6358, 6382, 6390, 6391, 6419, 6464, 6483, 6488, 6489, 6496, 6515, 6522, 6525, 6537, 6544, 6548, 6552, 6553, 6588, 6594, 6599, 6600, 6602, 6607, 6608, 6619, 6648, 6649, 6660, 6661, 6673, 6678, 6683, 6688, 6699, 6700, 6707, 6729, 6746, 6752, 6760, 6769, 6799, 6800, 6808, 6809, 6814, 6817, 6821, 6823, 6826, 6835, 6841, 6847, 6862, 6864, 6865, 6868, 6877, 6880, 6884, 6894, 6903, 6907, 6910, 6919, 6920, 6924, 6930, 6932, 6938, 6953, 6959, 6963, 6964, 6967, 6974, 6976, 6977, 6983, 7004, 7007, 7008, 7011, 7023, 7026, 7027, 7029, 7034, 7049, 7052, 7056, 7060, 7072, 7086, 7090, 7096, 7100, 7101, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7187, 7194, 7209, 7211, 7213, 7226, 7227, 7233, 7238, 7246, 7249, 7272, 7278, 7311, 7313, 7324, 7330, 7335, 7347, 7349, 7366, 7382, 7401, 7404, 7416, 7417, 7421, 7425, 7431, 7439, 7452, 7462, 7479, 7480] and total not in [17, 48, 108, 111, 125, 132, 136, 150, 218, 255, 292, 326, 376, 395, 436, 442, 538, 564, 588, 599, 610, 611, 618, 627, 651, 665, 686, 696, 707, 714, 724, 734, 735, 766, 797, 799, 813, 824, 841, 941, 994, 1021, 1052, 1077, 1092, 1096, 1126, 1135, 1272, 1328, 1349, 1437, 1644, 1701, 1729, 1730, 1774, 1800, 1851, 1855, 1861, 1870, 1896, 1926, 1956, 1976, 1979, 1988, 1993, 2103, 2146, 2148, 2153, 2160, 2169, 2191, 2204, 2207, 2220, 2287, 2288, 2296, 2309, 2469, 2497, 2500, 2517, 2569, 2627, 2756, 2814, 2822, 2855, 2897, 3003, 3013, 3047, 3049, 3067, 3082, 3083, 3090, 3106, 3201, 3212, 3280, 3288, 3329, 3346, 3451, 3491, 3503, 3504, 3539, 3543, 3603, 3607, 3615, 3629, 3641, 3665, 3692, 3714, 3715, 3739, 3767, 3790, 3807, 3812, 3839, 3850, 3918, 3927, 4027, 4038, 4040, 4044, 4048, 4057, 4063, 4067, 4091, 4105, 4145, 4150, 4167, 4192, 4240, 4266, 4284, 4327, 4402, 4605, 4638, 4646, 4667, 4687, 4777, 4794, 4804, 4811, 4812, 4813, 4825, 4884, 4897, 4952, 5020, 5043, 5060, 5067, 5099, 5109, 5138, 5156, 5188, 5199, 5202, 5218, 5222, 5257, 5259, 5296, 5323, 5376, 5400, 5528, 5567, 5656, 5729, 5798, 5807, 5815, 5836, 5839, 5853, 5867, 5873, 5877, 5905, 5907, 5912, 5913, 5974, 6029, 6154, 6243, 6250, 6298, 6340, 6522, 6525, 6548, 6594, 6600, 6602, 6607, 6608, 6648, 6683, 6700, 6711, 6729, 6746, 6752, 6760, 6769, 6800, 6809, 6814, 6826, 6841, 6864, 6868, 6884, 6894, 6910, 6974, 6977, 7008, 7023, 7026, 7052, 7086, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7211, 7227, 7249, 7250, 7272, 7278, 7313, 7330, 7425, 7452]):
                    continue
    
                question = question.replace("news papers", "newspapers")

                refined_text = initialize.getEntity.preprocess_context(context, title)
                dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)
                if dataEntities:
                    initialize.export.dumpdata(dataEntities[0])

                    outputAnswer = initialize.qna.findanswer(question, numberOfPairs)
                    if outputAnswer in answer or answer in outputAnswer:
                        correct += 1
                        corrects.append(total)

                
                # if total % 8000 == 0:
                #     print(f"Accuracy: {100*correct/total}")
                #     print(f"Correct: {correct}, out of {total}")
                # else:
                #     print("Not Applicable - mfeesh entities")
            if kolo % 1000 == 0:
                print(f"Accuracy: {100*correct/total}")
                print(f"Correct: {correct}, out of {total}")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Accuracy: {100*correct/total}")
        print(f"Correct: {correct}, out of {total}")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    return correct, total, errors, corrects


if __name__ == "__main__":
    initialize = Main()

    # dataset = load_dataset("rajpurkar/squad")
    # # dataset = load_dataset("quac")

    # # train and validation
    # train = dataset['train']
    # validation = dataset['validation']

    startsWith = ["who ","what", "when ", "where ", "how many "]
    # b3dein what
    # x = QuestionStartsWith_Accuracy(initialize, train, startsWith)
    x = QuestionStartsWith_Accuracy(initialize, validation, startsWith)

    
    



 30%|███       | 3212/10570 [00:19<00:14, 524.49it/s]

Accuracy: 0.12861736334405144
Correct: 2, out of 1555


 44%|████▍     | 4673/10570 [00:19<00:05, 1076.62it/s]

Accuracy: 0.06828269033799932
Correct: 2, out of 2929
Accuracy: 0.05652911249293386
Correct: 2, out of 3538


 72%|███████▏  | 7576/10570 [00:19<00:00, 3141.10it/s]

Accuracy: 0.04149377593360996
Correct: 2, out of 4820


100%|██████████| 10570/10570 [00:20<00:00, 520.13it/s] 

Accuracy: 0.026698705112802028
Correct: 2, out of 7491
Errors:  []


In [3]:
print("Accuracy: ", 100* x[0]/x[1])
print("Errors: ", x[2])
print("Corrects: ", x[3])
print("corrects: ", len(x[3]))
# corrects: [51, 79, 96, 137, 144, 159, 167, 181, 244, 260, 281, 297, 299, 300, 306, 307, 319, 322, 330, 362, 393, 434, 441, 450, 483, 507, 531, 537, 540, 564, 586, 587, 616, 653, 657, 683, 761, 765, 768, 795, 806, 842, 855, 876, 908, 919, 925, 948, 953, 961, 962, 984, 1001, 1003, 1004, 1082, 1086, 1103, 1110, 1132, 1176, 1258, 1329, 1345, 1389, 1392, 1421, 1424, 1427, 1442, 1446, 1448, 1470, 1477, 1482, 1501, 1511, 1528, 1534, 1567, 1613, 1650, 1673, 1692, 1694, 1695, 1698, 1708, 1709, 1717, 1721, 1764, 1772, 1806, 1820, 1834, 1836, 1885, 1936, 1958, 1959, 1965, 1975, 1985, 1986, 2042, 2068, 2076, 2105, 2108, 2146, 2150, 2258, 2279, 2303, 2315, 2328, 2348, 2355, 2357, 2360, 2370, 2375, 2389, 2404, 2407, 2429, 2465, 2470, 2484, 2499, 2528, 2556, 2559, 2590, 2644, 2645, 2650, 2653, 2655, 2658, 2693, 2695, 2699, 2732, 2733, 2734, 2739, 2757, 2792, 2802, 2803, 2804, 2810, 2813, 2831, 2926, 2938, 2942, 3028, 3031, 3040, 3066, 3086, 3120, 3183, 3189, 3200, 3202, 3208, 3214, 3227, 3257, 3299, 3341, 3350, 3368, 3421, 3446, 3466, 3477, 3514, 3524, 3529, 3534, 3563, 3570, 3581, 3591, 3594, 3610, 3615, 3617, 3623, 3645, 3652, 3716, 3718, 3747, 3758, 3762, 3784, 3795, 3800, 3801, 3803, 3804, 3807, 3819, 3831, 3835, 3852, 3860, 3862, 3880, 3944, 3955, 4105, 4117, 4119, 4126, 4134, 4136, 4143, 4169, 4174, 4239, 4291, 4320, 4328, 4379, 4408, 4477, 4530, 4537, 4557, 4560, 4573, 4580, 4587, 4599, 4605, 4618, 4622, 4632, 4638, 4654, 4655, 4656, 4660, 4663, 4705, 4759, 4760, 4770, 4779, 4781, 4788, 4799, 4803, 4804, 4809, 4811, 4819, 4825, 4829, 4843, 4854, 4862, 4867, 4872, 4886, 4893, 4921, 4947, 4953, 4955, 4957, 4961, 4965, 4977, 5017, 5020, 5041, 5042, 5044, 5047, 5055, 5057, 5086, 5090, 5098, 5107, 5109, 5122, 5137, 5138, 5139, 5141, 5157, 5183, 5191, 5213, 5216, 5218, 5303, 5314, 5315, 5325, 5341, 5350, 5352, 5356, 5365, 5367, 5373, 5375, 5376, 5398, 5405, 5406, 5414, 5436, 5439, 5447, 5456, 5484, 5504, 5505, 5578, 5664, 5670, 5683, 5686, 5699, 5705, 5752, 5777, 5785, 5817, 5821, 5843, 5849, 5851, 5870, 5875, 5916, 5920, 5937, 6012, 6016, 6017, 6018, 6029, 6055, 6059, 6065, 6082, 6098, 6152, 6155, 6167, 6177, 6178, 6180, 6184, 6186, 6188, 6190, 6194, 6195, 6199, 6203, 6205, 6206, 6207, 6215, 6217, 6218, 6219, 6235, 6259, 6281, 6287, 6306, 6311, 6313, 6320, 6332, 6343, 6347, 6348, 6349, 6350, 6375, 6377, 6383, 6385, 6415, 6422, 6434, 6447, 6449, 6450, 6453, 6478, 6486, 6493, 6503, 6507, 6510, 6512, 6515, 6516, 6526, 6527, 6558, 6570, 6605, 6628, 6632, 6657, 6693, 6702, 6707, 6732, 6739, 6741, 6774, 6806, 6816, 6838, 6841, 6845, 6848, 6851, 6860, 6871, 6879, 6900, 6904, 6911, 6980, 7018, 7030, 7034, 7043, 7047, 7084, 7085, 7088, 7091, 7108, 7133, 7139, 7144, 7167, 7170, 7183, 7186, 7188, 7192, 7211, 7217, 7222, 7235, 7247, 7253, 7274, 7275, 7280, 7294, 7311, 7329, 7368, 7387, 7413, 7415, 7426, 7429, 7433, 7440, 7452, 7461, 7462, 7464, 7465, 7467, 7469, 7472, 7474, 7475, 7478, 7483, 7487, 7488, 7491, 7494, 7497, 7499, 7501, 7502, 7505, 7508, 7511, 7519, 7520, 7522, 7534, 7539, 7552, 7554, 7562, 7564, 7571, 7578, 7587, 7589, 7597, 7600, 7603, 7715, 7733, 7734, 7766, 7790, 7802, 7805, 7812, 7851, 7858, 7870, 7906, 7908, 7942, 7948, 7981, 7984, 7985, 7986, 7987, 7988, 7993, 8016, 8019, 8022, 8024, 8029, 8039, 8041, 8058, 8065, 8088, 8093, 8098, 8102, 8140, 8145, 8149, 8177, 8188, 8219, 8255, 8266, 8268, 8280, 8281, 8282, 8284, 8295, 8311, 8338, 8400, 8403, 8509, 8523, 8524, 8525, 8535, 8541, 8553, 8558, 8579, 8618, 8649, 8693, 8696, 8713, 8717, 8721, 8724, 8734, 8745, 8747, 8796, 8797, 8803, 8805, 8810, 8875, 8888, 8925, 8960, 8982, 8985, 8987, 9003, 9004, 9010, 9056, 9093, 9100, 9102, 9104, 9111, 9114, 9119, 9123, 9125, 9128, 9130, 9134, 9136, 9137, 9139, 9150, 9153, 9158, 9159, 9167, 9171, 9178, 9186, 9191, 9193, 9196, 9228, 9279, 9283, 9293, 9324, 9356, 9359, 9371, 9372, 9378, 9379, 9391, 9452, 9453, 9457, 9458, 9460, 9463, 9469, 9510, 9511, 9523, 9524, 9539, 9575, 9576, 9577, 9581, 9591, 9595, 9616, 9617, 9618, 9620, 9621, 9622, 9654, 9663, 9665, 9667, 9668, 9669, 9679, 9682, 9683, 9709, 9736, 9756, 9761, 9763, 9767, 9833, 9838, 9844, 9847, 9852, 9862, 9878, 9894, 9906, 9911, 9920, 9921, 9926, 9945, 9946, 9948, 9955, 9956, 9974, 9982, 9985, 9997, 10001, 10004, 10033, 10055, 10056, 10071, 10088, 10098, 10123, 10139, 10140, 10162, 10185, 10186, 10189, 10216, 10226, 10255, 10302, 10326, 10340, 10341, 10342, 10343, 10354, 10373, 10385, 10420, 10477, 10547, 10568, 10593, 10594, 10610, 10613, 10614, 10618, 10624, 10653, 10654, 10662, 10664, 10671, 10675, 10776, 10788, 10800, 10801, 10807, 10810, 10857, 10858, 10881, 10892, 10899, 10901, 10910, 10916, 10944, 10967, 10971, 10972, 10984, 11041, 11086, 11090, 11091, 11105, 11116, 11139, 11172, 11191, 11193, 11220, 11236, 11258, 11262, 11273, 11290, 11297, 11308, 11309, 11313, 11316, 11324, 11340, 11349, 11362, 11368, 11369, 11377, 11413, 11490, 11552, 11619, 11623, 11663, 11678, 11747, 11762, 11783, 11788, 11816, 11849, 11869, 11891, 11897, 11900, 11902, 11966, 11980, 11995, 12032, 12052, 12078, 12094, 12095, 12122, 12131, 12135, 12141, 12148, 12156, 12206, 12209, 12212, 12236, 12281, 12312, 12317, 12325, 12341, 12357, 12358, 12394, 12410, 12411, 12418, 12425, 12473, 12514, 12525, 12530, 12550, 12570, 12596, 12597, 12603, 12625, 12631, 12645, 12651, 12652, 12719, 12729, 12738, 12746, 12750, 12754, 12779, 12780, 12797, 12799, 12804, 12818, 12848, 12867, 12893, 12902, 12913, 12915, 13075, 13079, 13080, 13095, 13118, 13130, 13143, 13144, 13145, 13152, 13155, 13172, 13174, 13189, 13193, 13206, 13299, 13308, 13322, 13330, 13331, 13337, 13348, 13350, 13363, 13365, 13392, 13394, 13427, 13441, 13474, 13495, 13522, 13547, 13549, 13598, 13672, 13696, 13733, 13778, 13805, 13806, 13822, 13836, 13849, 13856, 13859, 13861, 13954, 13955, 14026, 14052, 14054, 14058, 14086, 14122, 14132, 14133, 14144, 14185, 14187, 14192, 14193, 14200, 14205, 14212, 14213, 14215, 14225, 14233, 14295, 14296, 14307, 14317, 14330, 14349, 14350, 14361, 14365, 14383, 14395, 14396, 14417, 14421, 14422, 14427, 14439, 14440, 14445, 14447, 14452, 14453, 14456, 14474, 14488, 14504, 14511, 14532, 14555, 14619, 14628, 14634, 14655, 14698, 14724, 14749, 14753, 14856, 14862, 14865, 14874, 14914, 14924, 14935, 14967, 14980, 14993, 14999, 15010, 15013, 15017, 15019, 15040, 15052, 15098, 15105, 15113, 15151, 15154, 15156, 15193, 15212, 15252, 15259, 15266, 15275, 15295, 15301, 15303, 15306, 15314, 15316, 15319, 15322, 15327, 15349, 15382, 15388, 15447, 15476, 15484, 15501, 15518, 15519, 15524, 15537, 15548, 15551, 15571, 15588, 15598, 15619, 15650, 15666, 15679, 15697, 15725, 15727, 15755, 15757, 15769, 15771, 15782, 15809, 15822, 15823, 15834, 15858, 15866, 15879, 15880, 15882, 15885, 15886, 15887, 15894, 15895, 15896, 15925, 15934, 15949, 15952, 15973, 15996, 16031, 16053, 16090, 16093, 16101, 16103, 16105, 16133, 16150, 16172, 16212, 16214, 16217, 16218, 16245, 16249, 16288, 16305, 16307, 16314, 16354, 16381, 16437, 16440, 16442, 16562, 16593, 16599, 16604, 16647, 16653, 16657, 16661, 16690, 16728, 16736, 16744, 16766, 16817, 16847, 17061, 17072, 17075, 17076, 17090, 17174, 17234, 17236, 17262, 17269, 17272, 17371, 17376, 17377, 17433, 17451, 17507, 17530, 17564, 17705, 17731, 17732, 17771, 17782, 17783, 17796, 17799, 17806, 17807, 17809, 17822, 17835, 17850, 17855, 17863, 17877, 17886, 17915, 17930, 17937, 17957, 18027, 18052, 18056, 18073, 18090, 18092, 18093, 18095, 18097, 18107, 18110, 18115, 18116, 18118, 18128, 18130, 18133, 18137, 18140, 18141, 18149, 18153, 18168, 18173, 18205, 18234, 18353, 18374, 18383, 18398, 18411, 18450, 18454, 18455, 18461, 18484, 18496, 18500, 18583, 18590, 18639, 18701, 18754, 18755, 18807, 18818, 18823, 18830, 18832, 18920, 18921, 18960, 18999, 19039, 19042, 19044, 19082, 19083, 19126, 19151, 19164, 19165, 19169, 19179, 19228, 19238, 19244, 19264, 19273, 19294, 19325, 19328, 19329, 19335, 19396, 19442, 19445, 19463, 19494, 19501, 19511, 19542, 19617, 19620, 19631, 19635, 19656, 19685, 19724, 19744, 19782, 19878, 19898, 19924, 19927, 19939, 19943, 19951, 19954, 19967, 19968, 19971, 19987, 19989, 20003, 20025, 20026, 20041, 20044, 20069, 20143, 20145, 20146, 20165, 20183, 20185, 20190, 20209, 20214, 20224, 20230, 20249, 20286, 20330, 20336, 20402, 20418, 20428, 20454, 20472, 20513, 20516, 20521, 20530, 20531, 20547, 20567, 20660, 20665, 20685, 20702, 20713, 20730, 20751, 20777, 20791, 20792, 20862, 20922, 20932, 20957, 20985, 20988, 21014, 21023, 21026, 21032, 21037, 21038, 21112]
# dh kda mn 8eir what

# mn el vaidation: (msh bnbos root == root fl how many)
# Corrects:  [17, 48, 49, 59, 108, 111, 125, 132, 136, 150, 193, 218, 246, 251, 255, 258, 260, 268, 292, 326, 376, 395, 436, 442, 538, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 651, 665, 686, 696, 707, 714, 724, 734, 735, 766, 797, 799, 813, 824, 841, 927, 941, 994, 1021, 1052, 1077, 1092, 1096, 1126, 1135, 1272, 1286, 1328, 1349, 1437, 1644, 1701, 1729, 1730, 1774, 1800, 1851, 1855, 1861, 1870, 1896, 1926, 1956, 1976, 1979, 1988, 1993, 2103, 2146, 2148, 2153, 2160, 2169, 2170, 2191, 2204, 2207, 2220, 2287, 2288, 2296, 2309, 2469, 2497, 2500, 2517, 2569, 2627, 2756, 2814, 2822, 2855, 2897, 3003, 3013, 3047, 3049, 3067, 3082, 3083, 3090, 3106, 3212, 3280, 3288, 3329, 3346, 3451, 3491, 3503, 3504, 3539, 3543, 3603, 3607, 3615, 3629, 3641, 3665, 3692, 3714, 3715, 3739, 3767, 3790, 3807, 3812, 3839, 3842, 3850, 3918, 3927, 4027, 4038, 4040, 4044, 4048, 4057, 4063, 4067, 4091, 4105, 4145, 4150, 4167, 4192, 4240, 4266, 4284, 4327, 4402, 4605, 4638, 4646, 4667, 4687, 4777, 4794, 4804, 4811, 4812, 4813, 4825, 4884, 4897, 4952, 5020, 5043, 5060, 5067, 5099, 5109, 5138, 5156, 5188, 5199, 5202, 5218, 5222, 5257, 5259, 5296, 5323, 5376, 5400, 5503, 5528, 5532, 5559, 5560, 5567, 5656, 5729, 5798, 5807, 5815, 5836, 5839, 5853, 5867, 5873, 5877, 5905, 5907, 5912, 5913, 5974, 6029, 6154, 6175, 6243, 6250, 6298, 6340, 6522, 6525, 6548, 6594, 6600, 6602, 6607, 6608, 6648, 6683, 6700, 6711, 6729, 6746, 6752, 6760, 6769, 6800, 6809, 6814, 6826, 6841, 6864, 6868, 6884, 6894, 6910, 6974, 6977, 7008, 7023, 7026, 7052, 7086, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7211, 7227, 7249, 7250, 7272, 7278, 7313, 7330, 7425, 7452]
# Errors:  [3201]       # handled 5las

# dh b3d ma 7atet el condition bta3 root = root
# [17, 48, 108, 111, 125, 132, 136, 150, 218, 255, 292, 326, 376, 395, 436, 442, 538, 564, 588, 599, 610, 611, 618, 627, 651, 665, 686, 696, 707, 714, 724, 734, 735, 766, 797, 799, 813, 824, 841, 941, 994, 1021, 1052, 1077, 1092, 1096, 1126, 1135, 1272, 1328, 1349, 1437, 1644, 1701, 1729, 1730, 1774, 1800, 1851, 1855, 1861, 1870, 1896, 1926, 1956, 1976, 1979, 1988, 1993, 2103, 2146, 2148, 2153, 2160, 2169, 2191, 2204, 2207, 2220, 2287, 2288, 2296, 2309, 2469, 2497, 2500, 2517, 2569, 2627, 2756, 2814, 2822, 2855, 2897, 3003, 3013, 3047, 3049, 3067, 3082, 3083, 3090, 3106, 3201, 3212, 3280, 3288, 3329, 3346, 3451, 3491, 3503, 3504, 3539, 3543, 3603, 3607, 3615, 3629, 3641, 3665, 3692, 3714, 3715, 3739, 3767, 3790, 3807, 3812, 3839, 3850, 3918, 3927, 4027, 4038, 4040, 4044, 4048, 4057, 4063, 4067, 4091, 4105, 4145, 4150, 4167, 4192, 4240, 4266, 4284, 4327, 4402, 4605, 4638, 4646, 4667, 4687, 4777, 4794, 4804, 4811, 4812, 4813, 4825, 4884, 4897, 4952, 5020, 5043, 5060, 5067, 5099, 5109, 5138, 5156, 5188, 5199, 5202, 5218, 5222, 5257, 5259, 5296, 5323, 5376, 5400, 5528, 5567, 5656, 5729, 5798, 5807, 5815, 5836, 5839, 5853, 5867, 5873, 5877, 5905, 5907, 5912, 5913, 5974, 6029, 6154, 6243, 6250, 6298, 6340, 6522, 6525, 6548, 6594, 6600, 6602, 6607, 6608, 6648, 6683, 6700, 6711, 6729, 6746, 6752, 6760, 6769, 6800, 6809, 6814, 6826, 6841, 6864, 6868, 6884, 6894, 6910, 6974, 6977, 7008, 7023, 7026, 7052, 7086, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7211, 7227, 7249, 7250, 7272, 7278, 7313, 7330, 7425, 7452]

# dh lw shelt el condition menhom kolohom
# Corrects:  [17, 48, 49, 59, 87, 93, 108, 110, 111, 125, 129, 132, 150, 189, 193, 218, 234, 241, 245, 246, 251, 253, 255, 257, 258, 260, 267, 268, 270, 290, 292, 295, 298, 323, 326, 331, 334, 337, 339, 341, 369, 376, 385, 395, 436, 442, 479, 485, 503, 511, 517, 519, 521, 538, 543, 549, 559, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 643, 651, 665, 682, 686, 696, 707, 709, 714, 724, 729, 734, 735, 766, 795, 796, 797, 799, 808, 813, 815, 824, 841, 846, 866, 867, 871, 886, 892, 903, 905, 906, 915, 917, 923, 925, 927, 934, 935, 941, 946, 953, 956, 963, 964, 979, 985, 986, 987, 988, 994, 996, 999, 1001, 1003, 1006, 1008, 1010, 1016, 1019, 1021, 1024, 1039, 1052, 1063, 1077, 1096, 1099, 1111, 1114, 1126, 1132, 1135, 1137, 1142, 1148, 1152, 1154, 1164, 1165, 1166, 1171, 1176, 1178, 1188, 1192, 1198, 1199, 1201, 1206, 1209, 1216, 1230, 1237, 1239, 1249, 1272, 1279, 1280, 1286, 1287, 1291, 1299, 1304, 1308, 1313, 1314, 1315, 1328, 1349, 1354, 1372, 1414, 1437, 1450, 1457, 1468, 1477, 1494, 1496, 1521, 1545, 1546, 1552, 1569, 1604, 1609, 1617, 1621, 1625, 1628, 1637, 1642, 1644, 1672, 1676, 1678, 1683, 1690, 1695, 1696, 1698, 1700, 1701, 1702, 1704, 1705, 1708, 1714, 1728, 1729, 1730, 1749, 1751, 1763, 1765, 1768, 1774, 1777, 1778, 1782, 1786, 1789, 1792, 1797, 1799, 1800, 1816, 1818, 1830, 1831, 1841, 1846, 1851, 1855, 1860, 1861, 1870, 1871, 1887, 1891, 1893, 1896, 1905, 1917, 1924, 1926, 1929, 1932, 1937, 1938, 1939, 1947, 1948, 1952, 1956, 1961, 1972, 1976, 1979, 1982, 1983, 1990, 1992, 1993, 1999, 2003, 2006, 2030, 2042, 2088, 2089, 2103, 2132, 2134, 2145, 2146, 2148, 2153, 2156, 2160, 2169, 2170, 2175, 2181, 2191, 2197, 2200, 2201, 2204, 2207, 2211, 2220, 2252, 2254, 2260, 2268, 2272, 2274, 2287, 2288, 2292, 2296, 2309, 2313, 2314, 2341, 2378, 2419, 2431, 2451, 2452, 2469, 2490, 2497, 2500, 2510, 2517, 2526, 2552, 2553, 2560, 2569, 2609, 2610, 2612, 2613, 2623, 2627, 2631, 2639, 2655, 2663, 2665, 2678, 2679, 2683, 2685, 2710, 2712, 2725, 2734, 2756, 2762, 2765, 2781, 2788, 2789, 2797, 2798, 2814, 2822, 2826, 2843, 2851, 2855, 2872, 2891, 2893, 2897, 2910, 2960, 2974, 3003, 3013, 3014, 3018, 3041, 3043, 3044, 3047, 3049, 3063, 3064, 3067, 3068, 3080, 3082, 3083, 3088, 3090, 3095, 3106, 3201, 3209, 3212, 3216, 3247, 3252, 3257, 3263, 3280, 3284, 3288, 3292, 3315, 3318, 3325, 3329, 3333, 3346, 3348, 3374, 3396, 3397, 3465, 3475, 3480, 3482, 3487, 3491, 3503, 3504, 3524, 3543, 3603, 3607, 3615, 3625, 3629, 3649, 3665, 3666, 3684, 3692, 3696, 3702, 3715, 3739, 3751, 3767, 3790, 3797, 3799, 3801, 3807, 3812, 3826, 3835, 3839, 3842, 3850, 3918, 3921, 3923, 3927, 3958, 3960, 3979, 3989, 3993, 3997, 3999, 4006, 4013, 4023, 4027, 4029, 4036, 4038, 4040, 4044, 4048, 4049, 4057, 4063, 4067, 4069, 4080, 4084, 4091, 4092, 4096, 4097, 4098, 4105, 4107, 4111, 4117, 4126, 4130, 4144, 4145, 4150, 4155, 4160, 4167, 4181, 4184, 4186, 4192, 4202, 4204, 4205, 4240, 4246, 4253, 4266, 4271, 4282, 4284, 4295, 4297, 4314, 4319, 4327, 4340, 4367, 4398, 4402, 4403, 4406, 4419, 4423, 4425, 4427, 4431, 4447, 4450, 4457, 4528, 4533, 4541, 4544, 4557, 4573, 4575, 4590, 4592, 4598, 4605, 4638, 4646, 4651, 4658, 4667, 4687, 4707, 4750, 4787, 4794, 4799, 4801, 4812, 4813, 4825, 4830, 4884, 4897, 4929, 4941, 4951, 4952, 4953, 4984, 4987, 5020, 5031, 5043, 5060, 5063, 5067, 5080, 5099, 5100, 5107, 5109, 5138, 5147, 5156, 5160, 5175, 5188, 5189, 5199, 5200, 5202, 5208, 5218, 5222, 5224, 5229, 5234, 5239, 5257, 5259, 5262, 5279, 5281, 5296, 5308, 5323, 5330, 5346, 5349, 5350, 5377, 5382, 5393, 5400, 5408, 5414, 5427, 5448, 5462, 5470, 5474, 5497, 5499, 5502, 5503, 5518, 5526, 5527, 5528, 5532, 5534, 5559, 5560, 5562, 5567, 5568, 5569, 5570, 5594, 5595, 5640, 5642, 5646, 5647, 5656, 5660, 5667, 5669, 5677, 5702, 5707, 5709, 5722, 5729, 5755, 5761, 5763, 5769, 5775, 5780, 5784, 5788, 5798, 5800, 5802, 5806, 5807, 5809, 5815, 5819, 5834, 5836, 5839, 5840, 5851, 5853, 5867, 5870, 5873, 5877, 5878, 5886, 5894, 5900, 5903, 5905, 5907, 5912, 5913, 5921, 5947, 5970, 5974, 5984, 6016, 6029, 6033, 6057, 6059, 6060, 6061, 6085, 6086, 6088, 6089, 6093, 6100, 6105, 6125, 6135, 6138, 6140, 6146, 6154, 6160, 6161, 6168, 6175, 6188, 6191, 6227, 6243, 6244, 6250, 6251, 6252, 6253, 6271, 6273, 6279, 6281, 6286, 6298, 6311, 6314, 6356, 6358, 6382, 6390, 6391, 6419, 6464, 6483, 6488, 6489, 6496, 6515, 6522, 6525, 6537, 6544, 6548, 6552, 6553, 6588, 6594, 6599, 6600, 6602, 6607, 6608, 6619, 6648, 6649, 6660, 6661, 6673, 6678, 6683, 6688, 6699, 6700, 6707, 6729, 6746, 6752, 6760, 6769, 6799, 6800, 6808, 6809, 6814, 6817, 6821, 6823, 6826, 6835, 6841, 6847, 6862, 6864, 6865, 6868, 6877, 6880, 6884, 6894, 6903, 6907, 6910, 6919, 6920, 6924, 6930, 6932, 6938, 6953, 6959, 6963, 6964, 6967, 6974, 6976, 6977, 6983, 7004, 7007, 7008, 7011, 7023, 7026, 7027, 7029, 7034, 7049, 7052, 7056, 7060, 7072, 7086, 7090, 7096, 7100, 7101, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7187, 7194, 7209, 7211, 7213, 7226, 7227, 7233, 7238, 7246, 7249, 7272, 7278, 7311, 7313, 7324, 7330, 7335, 7347, 7349, 7366, 7382, 7401, 7404, 7416, 7417, 7421, 7425, 7431, 7439, 7452, 7462, 7479, 7480]
# Accuracy:  11.640635429181685
# corrects:  872

Accuracy:  11.640635429181685
Errors:  []
Corrects:  [17, 48, 49, 59, 87, 93, 108, 110, 111, 125, 129, 132, 150, 189, 193, 218, 234, 241, 245, 246, 251, 253, 255, 257, 258, 260, 267, 268, 270, 290, 292, 295, 298, 323, 326, 331, 334, 337, 339, 341, 369, 376, 385, 395, 436, 442, 479, 485, 503, 511, 517, 519, 521, 538, 543, 549, 559, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 643, 651, 665, 682, 686, 696, 707, 709, 714, 724, 729, 734, 735, 766, 795, 796, 797, 799, 808, 813, 815, 824, 841, 846, 866, 867, 871, 886, 892, 903, 905, 906, 915, 917, 923, 925, 927, 934, 935, 941, 946, 953, 956, 963, 964, 979, 985, 986, 987, 988, 994, 996, 999, 1001, 1003, 1006, 1008, 1010, 1016, 1019, 1021, 1024, 1039, 1052, 1063, 1077, 1096, 1099, 1111, 1114, 1126, 1132, 1135, 1137, 1142, 1148, 1152, 1154, 1164, 1165, 1166, 1171, 1176, 1178, 1188, 1192, 1198, 1199, 1201, 1206, 1209, 1216, 1230, 1237, 1239, 1249, 1272, 1279, 1280, 1286, 1287, 1291, 1299, 1304, 1308, 1313, 1314, 1315, 1328, 1349, 1354,

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
# dataset = load_dataset("quac")

# train and validation
train = dataset['train']
validation = dataset['validation']

# save them to csv
# train.to_csv('dataset/train.csv')
# validation.to_csv('dataset/validation.csv')
